In [25]:
import sympy as sym
import sympy.printing as printing
MAGNETIC_PERMEABILITY = 12.57e-7
TtoAm = 795774.715459


# Symbolic computation

In [187]:
theta, phi, lamb, omega = sym.symbols('Theta Phi Lambda Omega')
alpha1, alpha2, gamma = sym.symbols("alpha_1 alpha_2 gamma")
torque_par, torque_per = sym.symbols("tau_1 tau_2")
K1, K2 = sym.symbols("K1 K2")
Hoe = sym.symbols("H_{oe}")

"""
Theta, azimuth L1
Phi, inplane L1

Lambda, azimuth L1
Omega, inplane L2
"""

sinT = sym.sin(theta)
cosT = sym.cos(theta)

cosL = sym.cos(lamb)
sinL = sym.sin(lamb)

sinO = sym.sin(omega)
cosO = sym.cos(omega)

sinP = sym.sin(phi)
cosP = sym.cos(phi)

cosPO = sym.cos(phi - omega)
sinPO = sym.sin(phi - omega)


Ms1, Ms2 = sym.symbols("M_{s1} M_{s2}")

const1 = (gamma / Ms1) * (1 / (1 + alpha1**2))
const2 = (gamma / Ms2) * (1 / (1 + alpha2**2))


Ho1, Ho2, Ho3 = sym.symbols("H_{oe1} H_{oe2} H_{oe3}")
He1, He2, He3 = sym.symbols("H_{ext1} H_{ext2} H_{ext3}")
Hd11, Hd12, Hd13 = sym.symbols("H_{dip11} H_{dip12} H_{dip13}")
Hd21, Hd22, Hd23 = sym.symbols("H_{dip21} H_{dip22} H_{dip23}")
Hde11, Hde12, Hde13 = sym.symbols("H_{de11} H_{de12} H_{de13}")
Hde21, Hde22, Hde23 = sym.symbols("H_{de21} H_{de22} H_{de23}")

Hoe = sym.Matrix([Ho1, Ho2, Ho3])
Hext = sym.Matrix([He1, He2, He3])
Hdip1 = sym.Matrix([Hd11, Hd12, Hd13])
Hdip2 = sym.Matrix([Hd21, Hd22, Hd23])
Hdemag1 = sym.Matrix([Hde11, Hde12, Hde13])
Hdemag2 = sym.Matrix([Hde21, Hde22, Hde23])

M1 = sym.Matrix([Ms1 * sinT * cosP, Ms1 * sinT * sinP, Ms1 * cosT])
M2 = sym.Matrix([Ms2 * sinL * cosO, Ms2 * sinL * sinO, Ms2 * cosL])

# Hext = sym.MatrixSymbol("H_{ext}", 1, 3)
# Hdip1 = sym.MatrixSymbol("H_{dip1}", 1, 3)
# Hdip2 = sym.MatrixSymbol("H_{dip2}", 1, 3)
# Hdemag1 = sym.MatrixSymbol("H_{demag1}", 1, 3)
# Hdemag2 = sym.MatrixSymbol("H_{demag2}", 1, 3)

# M1 = sym.MatrixSymbol("M_{s1}", 1, 3)
# M2 = sym.MatrixSymbol("M_{s2}", 1, 3)



anisotropies = K1 * (cosT**2 + (sinT**2) *
                     (sinP**2)) + K2 * (cosL**2 + (sinL**2) * (sinO**2))
dipole = -M1.dot(Hdip2) - M2.dot(Hdip1)
demag = -M1.dot(Hdemag1) - M2.dot(Hdemag2)
ext = -M1.dot(Hext) - M2.dot(Hext)
"""
Can't figure out the part below for the Matrix Symbol
"""
# dipole = -M1*Hdip2.T - M2*Hdip1.T
# demag = -M1*Hdemag1.T - M2*Hdemag2.T
# ext = -M1*Hext.T - M2*Hext.T

energy = anisotropies + dipole + demag + ext
dUdP = sym.diff(energy, phi)
dUdT = sym.diff(energy, theta)
dUdO = sym.diff(energy, omega)
dUdL = sym.diff(energy, lamb)

In [100]:
dTheta = const1 * ((-1 / sinT)*dUdP - alpha1*dUdT -
                  (cosL * sinT - cosT * sinL * cosPO) *
                  (torque_par + alpha1 * torque_per) - (sinL * sinPO) *
                  (alpha1 * torque_par - torque_per) + Ms1*Hoe)

In [91]:
dPhi = const1 * ((1 / sinT) * dUdT - (alpha1 / (sinT**2)) * dUdP + (1 / sinT) *
                (cosL * sinT - cosT * sinL * cosPO) *
                (alpha1 * torque_par - torque_per) 
                - (1/sinT)*(sinL*sinPO)*(torque_par + alpha1*torque_per) + alpha1*Ms1*Hoe/sinT)

In [92]:
dLam = const2 * ((-1 / sinT)*dUdO - alpha2*dUdL +
                  (-sinL * cosT + sinT * cosL * cosPO) *
                  (torque_par + alpha2 * torque_per) + (sinL * sinPO) *
                  (alpha2 * torque_par + torque_per) + Ms2*Hoe) 

In [93]:
dOmega = const2 * ((1 / sinT) * dUdL - (alpha2 / (sinL**2)) * dUdO - (1 / sinL) *
                (sinL * cosT - sinT *cosL * cosPO) *
                (torque_per - alpha2 * torque_par) 
                + (1/sinL)*(sinT*sinPO)*(torque_par - alpha2*torque_per) + alpha2*Ms2*Hoe/sinL)

## The derivative matrix 

In [122]:
dM = sym.Matrix([
    [
        sym.diff(dTheta, theta),
        sym.diff(dTheta, phi),
        sym.diff(dTheta, lamb),
        sym.diff(dTheta, omega)
    ],
    [
        sym.diff(dPhi, theta),
        sym.diff(dPhi, phi),
        sym.diff(dPhi, lamb),
        sym.diff(dPhi, omega)
    ],
    [
        sym.diff(dLam, theta),
        sym.diff(dLam, phi),
        sym.diff(dLam, lamb),
        sym.diff(dLam, omega)
    ],
    [
        sym.diff(dOmega, theta),
        sym.diff(dOmega, phi),
        sym.diff(dOmega, lamb),
        sym.diff(dOmega, omega)
    ]]
)

In [170]:
"""
This contains also the parameter order
"""
sub = {
    theta: 1,
    phi: 2,
    lamb: 3,
    omega: 4,
    # Hoe
    Hoe: 2,
    # Hext
    He1: 1,
    He2: 2,
    He3: 1,
    # dipole 1
    Hd11: 2,
    Hd12: 3,
    Hd13: 1,
    # dipole 2
    Hd21: 2,
    Hd22: 3,
    Hd23: 1,
    # demag 1 
    Hde11: 3,
    Hde12: 1,
    Hde13: 1,
    # demag 2
    Hde21: 3,
    Hde22: 1,
    Hde23: 1,
    gamma: 2.21e5,
    alpha1: 0.035,
    alpha2: 0.035,
    torque_par: 1.0,
    torque_per: 1.0,
    Ms1: 1 * TtoAm,
    Ms2: 1 * TtoAm,
    K1: 350e3,
    K2: 650e3
}

dM.evalf(subs=sub)

Matrix([
[ 7449.77426608095,  -577180.312268086,  0.338949450439031, -0.00418479210239431],
[ 985772.506426296,   1025004.48517294, -0.198540949002376,  -0.0420899406931635],
[-12001.0454815646, -0.234323771239703,  -64155.5729692391,     311521.739498639],
[ 1323372.57650519,   -2.1009604200395,  -262336.979320066,       566627.2453772]])

In [171]:
dMFunction = sym.lambdify([list(sub.keys())], dM, "numpy")

## The energy function and gradient

In [114]:
energy

-H_{de11}*M_{s1}*sin(Theta)*cos(Phi) - H_{de12}*M_{s1}*sin(Phi)*sin(Theta) - H_{de13}*M_{s1}*cos(Theta) - H_{de21}*M_{s2}*sin(Lambda)*cos(Omega) - H_{de22}*M_{s2}*sin(Lambda)*sin(Omega) - H_{de23}*M_{s2}*cos(Lambda) - H_{dip11}*M_{s2}*sin(Lambda)*cos(Omega) - H_{dip12}*M_{s2}*sin(Lambda)*sin(Omega) - H_{dip13}*M_{s2}*cos(Lambda) - H_{dip21}*M_{s1}*sin(Theta)*cos(Phi) - H_{dip22}*M_{s1}*sin(Phi)*sin(Theta) - H_{dip23}*M_{s1}*cos(Theta) - H_{ext1}*M_{s1}*sin(Theta)*cos(Phi) - H_{ext1}*M_{s2}*sin(Lambda)*cos(Omega) - H_{ext2}*M_{s1}*sin(Phi)*sin(Theta) - H_{ext2}*M_{s2}*sin(Lambda)*sin(Omega) - H_{ext3}*M_{s1}*cos(Theta) - H_{ext3}*M_{s2}*cos(Lambda) + K1*(sin(Phi)**2*sin(Theta)**2 + cos(Theta)**2) + K2*(sin(Lambda)**2*sin(Omega)**2 + cos(Lambda)**2)

In [172]:
energy.evalf(subs=sub)

994119.049798854

In [173]:
# a vector of energy derivatives w.r.t to each of the angles
energy_gradient = sym.Matrix([
    sym.diff(energy, theta),
    sym.diff(energy, phi),
    sym.diff(energy, lamb),
    sym.diff(energy, omega),
])
energy_gradient.evalf(subs=sub)

Matrix([
[ 681542.379191033],
[ 5137719.42530403],
[-6252493.39216735],
[-56701.3821265716]])

In [174]:
energyFunction = sym.lambdify([list(sub.keys())], energy, "numpy")
energyGradientFunction = sym.lambdify([list(sub.keys())], energy_gradient, "numpy")

In [175]:
energyGradientFunction(list(x0.values())+list(arg_set.values()))

array([[ 3886829.64422199],
       [ 4475256.04184709],
       [  574173.95946033],
       [-3588442.63996175]])

### TODO: 
Make all the arguments in the lambdified function named

## Numerical optimisations

In [176]:
import numpy as np
from scipy.optimize import minimize

MAGNETIC_PERMEABILITY = 12.57e-7
TtoAm = 795774.715459


def opt_wrapper_cost(x, args):
    theta, phi, lamb, omega = x
    return energyFunction([theta, phi, lamb, omega] + args)

def opt_wrapper_grad(x, args):
    theta, phi, lamb, omega = x
    return energyGradientFunction([theta, phi, lamb, omega] + args)

def to_radian(angle):
    return angle * (np.pi / 180)


def get_tensor_interaction(M, N):
    return N @ M


def cartiesian_from_spherical(r, inplane, azimuth):
    return np.array([
        r * np.sin(azimuth) * np.cos(inplane),
        r * np.sin(azimuth) * np.sin(inplane), r * np.cos(azimuth)
    ])


"""
        Spherical coords
        Theta -- azimuth angle, L1
        Phi -- inplane angle, L1
        Lambda -- azimuth angle, L2
        Omega -- inplane angle, L2
"""

x0 = {
    theta: to_radian(90),
    phi: to_radian(45),
    lamb: to_radian(90),
    omega: to_radian(45),
}


arg_set = {
    # Hoe
    Hoe: 397.88,
    # Hext
    He1: 1,
    He2: 2,
    He3: 1,
    # dipole 1
    Hd11: 0,
    Hd12: 0,
    Hd13: 0,
    # dipole 2
    Hd21: 0,
    Hd22: 0,
    Hd23: 0,
    # demag 1 
    Hde11: 0,
    Hde12: 0,
    Hde13: 1,
    # demag 2
    Hde21: 0,
    Hde22: 0,
    Hde23: 1,
    gamma: 2.21e5,
    alpha1: 0.035,
    alpha2: 0.035,
    torque_par: 0.0,
    torque_per: 0.0,
    Ms1: 1 * TtoAm,
    Ms2: 1 * TtoAm,
    K1: 728e3,
    K2: 305e3
}

bounds = [(0, np.pi * 2) for i in range(len(x0))]
res = minimize(opt_wrapper_cost,
               list(x0.values()),
               bounds=bounds,
               args=list(arg_set.values()),
               jac=opt_wrapper_grad,
               tol=1e-4)

In [184]:
matrix_res = dMFunction(res.x.tolist() + list(arg_set.values()))
eigenvalues, eivenvectors = np.linalg.eig(matrix_res)

In [185]:
eigenvalues

array([ -943661.92414909,   898153.88773187, -1042274.41255696,
         998869.16657767])

In [186]:
eivenvectors

array([[-5.14638251e-01, -4.21761714e-01, -6.76996627e-18,
        -2.36439414e-18],
       [-8.57407290e-01,  9.06706695e-01, -1.55676519e-17,
         6.27383792e-18],
       [ 1.72752180e-04, -4.63404615e-05,  3.89829426e-01,
        -3.59389194e-01],
       [ 4.24928988e-04,  1.56707168e-04,  9.20887082e-01,
         9.33187766e-01]])